Author: Owen Burt  
Date created: 2/21/2025  
Description:  
References:  
Sources:  

In [1]:
import tensorflow as tf
import pandas as pd
import cv2
import os as os
import glob

### Creating training and testing data
I need to convert my images to arrays and make an array for labels and bounds.  
The bounds will be combined with labels creating the target.
I need to somehow create training and testing splits in the data.  
Also I need to form a dataset with these arrays.  
What does that look like????

In [2]:
cwd = os.getcwd()
print(cwd)

c:\Users\owend\Eye_tracker\Eye_tracker


In [3]:
df = pd.read_csv(cwd + "/eye_data.csv")

In [4]:
df

,file_name,obj_class,x1,y1,x2,y2
0,WIN_20250205_16_00_55_Pro.jpg,pupil,257,163,276,195
1,WIN_20250205_16_00_56_Pro.jpg,pupil,279,128,308,156
2,WIN_20250205_16_00_57_Pro.jpg,pupil,313,138,346,170
3,WIN_20250205_16_00_58_Pro.jpg,pupil,278,141,305,172
4,WIN_20250205_16_00_59_Pro.jpg,pupil,263,150,286,181
...,...,...,...,...,...,...
363,WIN_20250220_20_01_15_Pro.jpg,NEGATIVE,0,0,0,0
364,WIN_20250220_20_01_17_Pro.jpg,NEGATIVE,0,0,0,0
365,WIN_20250220_20_01_19_Pro.jpg,NEGATIVE,0,0,0,0
366,WIN_20250220_20_01_28_Pro.jpg,NEGATIVE,0,0,0,0


In [5]:
from tensorflow.keras.preprocessing import image

In [7]:
img = image.load_img((cwd + "/anotated_images/" + df.iloc[0][0]), target_size=(640, 360, 3))

C:\Users\owend\AppData\Local\Temp\ipykernel_20036\2162205132.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img = image.load_img((cwd + "/anotated_images/" + df.iloc[0][0]), target_size=(640, 360, 3))


In [ ]:
images = []
images.append(image.img_to_array(img))

In [10]:
df.iloc[0][0]

C:\Users\owend\AppData\Local\Temp\ipykernel_20036\2900375399.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[0][0]


'WIN_20250205_16_00_55_Pro.jpg'

In [ ]:
X_images = []
y_bounds = []
y_labels = []

for index, row in df.iterrows():
    
    img = image.load_img(cwd + "/all_images/" + row["file_name"], target_size=(640, 360, 3))
    X_images.append(image.img_to_array(img))

    y_bounds.append([row["x1"], row["y1"], row["x2"], row["y2"]])
    y_labels.append(row["obj_class"])


In [24]:
dataset = tf.data.Dataset.from_tensor_slices((X_images, (y_bounds, y_labels)))